# PANTHER API access and gene annotation

Functional annotation of protein families in Differentially Expressed Genes and Leading Edge list in Enrichment result

_Author : Natalia García Sánchez_

_Date : 25/05/2023_

---

In [1]:
import requests
import pandas as pd


current panther release

In [17]:
request_url =  "http://pantherdb.org/services/oai/pantherdb/supportedgenomes"
res = requests.post(request_url, {'caller_identity':'https://github.com/Natpod/Bnapus_ME'})

In [ ]:
version=res.json()['search']['product']


In [29]:
version

{'source': 'PANTHERDB', 'version': 17}

In [33]:
gen=res.json()['search']['output']['genomes']['genome']
supportedGenomes = pd.DataFrame.from_dict(gen)

In [36]:
supportedGenomes[supportedGenomes['taxon_id']==3708]

,name,taxon_id,short_name,version,long_name
26,canola,3708,BRANA,Reference Proteome 2021_03,Brassica napus


download genome annotations POST (only 1000 genes at a time)

In [41]:
start_index=1
## Construct the request
##
request_url = f"http://pantherdb.org/services/oai/pantherdb/downloadgenome?organism=3708&startIndex={start_index}"

params = {
    "caller_identity": "https://github.com/Natpod/Bnapus_ME",
}
results = requests.post(request_url, data=params)
#df = handle_results(results)

In [43]:
response = results.json()

In [46]:
print("Number of genes found for protein functional annotation:", response['search']['number_of_genes_in_genome'])

Number of genes found for protein functional annotation: 60104


In [313]:
final_index = response['search']['number_of_genes_in_genome']
max_genes_retrieved = 1000
num_of_requests = int(round((final_index-start_index)/max_genes_retrieved,0))

Do for all genes

In [323]:
final_df = pd.DataFrame(columns = [['gene_symbol','gene_name','persistent_id','name_PC','id_PC']])

start_index=1
final_index = response['search']['number_of_genes_in_genome']
max_genes_retrieved = 1000
num_of_requests = int(round((final_index-start_index)/max_genes_retrieved,0))

indexes_to_test = [1]
indexes_to_test = indexes_to_test + [i*1000 for i in range(1,num_of_requests+1)]

for st_index_request in indexes_to_test:
    ## Construct the request
    ##
    request_url = f"http://pantherdb.org/services/oai/pantherdb/downloadgenome?organism=3708&startIndex={st_index_request}"

    params = {
        "caller_identity": "https://github.com/Natpod/Bnapus_ME",
    }
    results = requests.post(request_url, data=params)






    gl_1 = pd.DataFrame.from_dict(response['search']['gene_list']['gene'])
    gl_1 = gl_1.replace(nanval,None)

    list_genes = gl_1['gene_symbol'].to_list()
    list_gene_names = gl_1['gene_name'].to_list()
    list_gene_pantherid = gl_1['persistent_id'].to_list()
    list_name_pc= []
    list_id_pc = []

    for i in range(len(list_annotations)):
        if list_annotations[i] != None:
            tmp_df = pd.DataFrame.from_dict(list_annotations[i]['annotation_data_type'])
            if tmp_df.shape[0] > 1 :
                if (tmp_df['content'] == 'ANNOT_TYPE_ID_PANTHER_PC').any():
                    tmp_df = tmp_df[tmp_df['content'] == 'ANNOT_TYPE_ID_PANTHER_PC']['annotation_list']
                    dict_pc = tmp_df.iloc[0]
                    #subdict = dict_pc.get('annotation')
                    name_pc = dict_pc.get('annotation').get('name')
                    id_pc = dict_pc.get('annotation').get('id')
            else:
                if (tmp_df['content'] == 'ANNOT_TYPE_ID_PANTHER_PC').any():
                    tmp_df = tmp_df[tmp_df['content'] == 'ANNOT_TYPE_ID_PANTHER_PC']['annotation_list']
                    name_pc = tmp_df.iloc[0].get('name')
                    id_pc = tmp_df.iloc[0].get('id')

            list_name_pc.append(name_pc)
            list_id_pc.append(id_pc)
        else:
            list_name_pc.append(None)
            list_id_pc.append(None)


    prepr_df = pd.DataFrame(list(zip(list_genes,list_gene_names,list_gene_pantherid,list_name_pc,list_id_pc)), columns = [['gene_symbol','gene_name','persistent_id','name_PC','id_PC']])
    final_df = pd.concat((final_df, prepr_df))

In [324]:
final_df.shape

(61000, 5)

Get _Brassica napus_ Hidden Markov Model Based assignation of PANTHERdb protein family classification on ftp browser on linux command (current PANTHER release version : v.17).The assignment is based on the phylogenetic trees where the gene is located.  The annotation of the subfamilies are used for the gene.

```
wget 'http://data.pantherdb.org/ftp/sequence_classifications/17.0/PANTHER_Sequence_Classification_files/PTHR17.0_rape'

```



In [343]:
df_Fam = pd.read_csv('/home/famgarcia/Descargas/PTHR17.0_rape', sep="\t", header=None)

df_Fam.rename(columns=dict(zip(df_Fam.columns,['gene_id','Uniprot_ID','gene_symbol','PANTHER_SF_ID','PANTHER_Family_name','PANTHER_SF_name','GO_MF','GO_BP','GO_CC','Protein_class','Pathway'])), inplace=True)
df_Fam = df_Fam[['Uniprot_ID','gene_symbol','PANTHER_SF_ID','PANTHER_Family_name','PANTHER_SF_name']]

In [338]:
df_Fam.head()

,Uniprot_ID,gene_symbol,PANTHER_SF_ID,PANTHER_Family_name,PANTHER_SF_name
0,A0A078JWU9,BnaAnng36540D,PTHR31929:SF143,SAUR-LIKE AUXIN-RESPONSIVE PROTEIN FAMILY-RELATED,BNAANNG36540D PROTEIN
1,A0A078IGF9,BnaC05g48700D,PTHR45621:SF151,OS01G0588500 PROTEIN-RELATED,PROTEIN KINASE DOMAIN-CONTAINING PROTEIN
2,A0A078FKQ7,BnaCnng04230D,PTHR21649:SF120,CHLOROPHYLL A/B BINDING PROTEIN,"CHLOROPHYLL A-B BINDING PROTEIN, CHLOROPLASTIC"
3,A0A078H4L9,BnaA06g35530D,PTHR15680:SF16,RIBOSOMAL PROTEIN L19,BNAA06G35530D PROTEIN
4,A0A078JN16,BnaUnng02600D,PTHR11945:SF688,MADS BOX PROTEIN,MADS-BOX DOMAIN-CONTAINING PROTEIN


In [346]:
final_df.head()

final_df.columns = final_df.columns.map(''.join)

merge both results

In [347]:
df_annotated = pd.merge(final_df, df_Fam, on="gene_symbol",how="outer")

In [353]:
df_annotated = df_annotated.drop_duplicates()

In [369]:
df_annotated

,gene_symbol,gene_name,persistent_id,name_PC,id_PC,Uniprot_ID,PANTHER_SF_ID,PANTHER_Family_name,PANTHER_SF_name
0,BnaC09g12820D,BnaC09g12820D protein,PTN008197571,DNA-binding transcription factor,PC00218,A0A078F5I3,PTHR46266:SF3,TRANSCRIPTION FACTOR TT8,TRANSCRIPTION FACTOR EGL1
1,BnaC09g12810D,Glutamine amidotransferase,PTN007548798,ligase,PC00142,A0A078F1H1,PTHR11922:SF2,GMP SYNTHASE-RELATED,GMP SYNTHASE [GLUTAMINE-HYDROLYZING]
2,BnaC09g12800D,BnaC09g12800D protein,PTN008228993,None,None,A0A078F4H7,PTHR47212:SF16,"ADHESIN-LIKE PROTEIN, PUTATIVE (DUF3741)-RELATED",BNAA09G12400D PROTEIN
3,BnaC09g12790D,BnaC09g12790D protein,PTN008654183,ligase,PC00142,A0A078F5D2,PTHR33248:SF42,ZINC ION-BINDING PROTEIN,GRF-TYPE DOMAIN-CONTAINING PROTEIN
4,BnaC09g12780D,BnaC09g12780D protein,PTN007561817,aspartic protease,PC00053,A0A078F968,PTHR12174:SF75,SIGNAL PEPTIDE PEPTIDASE,SIGNAL PEPTIDE PEPTIDASE-LIKE 2
...,...,...,...,...,...,...,...,...,...
43283,BnaC03g34740D,NaN,NaN,NaN,NaN,A0A078F4W9,PTHR11157:SF143,FATTY ACID ACYL TRANSFERASE-RELATED,VERY-LONG-CHAIN 3-OXOACYL-COA SYNTHASE
43284,BnaC09g43750D,NaN,NaN,NaN,NaN,A0A078F8V0,PTHR31901:SF43,GH3 DOMAIN-CONTAINING PROTEIN,4-SUBSTITUTED BENZOATES-GLUTAMATE LIGASE GH3.12
43285,BnaA09g27500D,NaN,NaN,NaN,NaN,A0A078IBM3,PTHR35996:SF1,OSJNBA0038O10.25 PROTEIN,OS04G0528100 PROTEIN
43286,BnaA04g14890D,NaN,NaN,NaN,NaN,A0A078HEX9,PTHR13318:SF88,UNCHARACTERIZED,EIN3-BINDING F-BOX PROTEIN 1


In [379]:
df_annotated= df_annotated.replace(nanval,None)

# Gene annotation

Annotating predicted class on Leading Edge (LE) genes

In [370]:
# Leading edge genes from singificantly enriched GSEA pathways
le_UP_HDAC = pd.read_csv('/home/famgarcia/le_HDAC_up.txt', header=None)
le_DOWN_HDAC = pd.read_csv('/home/famgarcia/le_HDAC_down.txt', header=None)

le_UP_STRESS = pd.read_csv('/home/famgarcia/le_Stress_up.txt', header=None)
le_DOWN_STRESS = pd.read_csv('/home/famgarcia/le_Stress_down.txt', header=None)

le_UP_HDAC.rename(columns={le_UP_HDAC.columns[0]:'gene_symbol'}, inplace=True)
le_DOWN_HDAC.rename(columns={le_DOWN_HDAC.columns[0]:'gene_symbol'}, inplace=True)
le_UP_STRESS.rename(columns={le_UP_STRESS.columns[0]:'gene_symbol'}, inplace=True)
le_DOWN_STRESS.rename(columns={le_DOWN_STRESS.columns[0]:'gene_symbol'}, inplace=True)


In [385]:
le_UP_HDAC_ann = pd.merge(le_UP_HDAC,df_annotated,on='gene_symbol',how='inner')

In [386]:
le_DOWN_HDAC_ann = pd.merge(le_DOWN_HDAC,df_annotated,on='gene_symbol',how='inner')

In [387]:
le_UP_STRESS_ann = pd.merge(le_UP_STRESS,df_annotated,on='gene_symbol',how='inner')

In [388]:
le_DOWN_STRESS_ann = pd.merge(le_DOWN_STRESS,df_annotated,on='gene_symbol',how='inner')

Annotating predicted class on Differentially Expressed Genes (DEGs)

In [391]:
# DEGS from both experiments
DEG_HDAC = pd.read_excel('/home/famgarcia/Filtered_DEG_LFC1_padj05_results_total.xlsx')
DEG_STRESS = pd.read_csv('/home/famgarcia/Final_condition_PErep_vs_VMrep_padj0.05_woPErep3_LFC1filtered.csv')

In [394]:
DEG_HDAC.rename(columns={'external_gene_name':'gene_symbol'},inplace=True)
DEG_STRESS.rename(columns={'external_gene_name':'gene_symbol'},inplace=True)

In [397]:
DEG_UP_STRESS = DEG_STRESS[DEG_STRESS['log2FoldChange']>0][['gene_symbol','log2FoldChange','padj']]
DEG_DOWN_STRESS = DEG_STRESS[DEG_STRESS['log2FoldChange']<0][['gene_symbol','log2FoldChange','padj']]

DEG_UP_HDAC = DEG_HDAC[DEG_HDAC['log2FoldChange']>0][['gene_symbol','log2FoldChange','padj']]
DEG_DOWN_HDAC = DEG_HDAC[DEG_HDAC['log2FoldChange']<0][['gene_symbol','log2FoldChange','padj']]

In [398]:
DEG_UP_STRESS_ann = pd.merge(DEG_UP_STRESS,df_annotated,on='gene_symbol',how='inner')
DEG_DOWN_STRESS_ann = pd.merge(DEG_DOWN_STRESS,df_annotated,on='gene_symbol',how='inner')

DEG_UP_HDAC_ann = pd.merge(DEG_UP_HDAC,df_annotated,on='gene_symbol',how='inner')
DEG_DOWN_HDAC_ann = pd.merge(DEG_DOWN_HDAC,df_annotated,on='gene_symbol',how='inner')

In [448]:
DEG_DOWN_STRESS_ann.to_csv('/home/famgarcia/TFM/DEGs_DOWN_STRESS_ann.tsv', sep="\t", index=False)
DEG_UP_STRESS_ann.to_csv('/home/famgarcia/TFM/DEGs_UP_STRESS_ann_ann.tsv', sep="\t", index=False)
DEG_UP_HDAC_ann.to_csv('/home/famgarcia/TFM/DEGs_UP_HDAC_ann.tsv', sep="\t", index=False)
DEG_DOWN_HDAC_ann.to_csv('/home/famgarcia/TFM/DEGs_DOWN_HDAC_ann.tsv', sep="\t", index=False)

Save annotations of DEGs that are in a leading edge in enrichment

In [446]:
tmp_df = pd.merge(le_DOWN_STRESS_ann,DEG_DOWN_STRESS,on='gene_symbol')
tmp_df.to_csv('/home/famgarcia/TFM/LE_DEGs_DOWN_STRESS.tsv', sep="\t", index=False)

In [447]:
tmp_df = pd.merge(le_UP_STRESS_ann,DEG_UP_STRESS,on='gene_symbol')
tmp_df.to_csv('/home/famgarcia/TFM/LE_DEGs_UP_STRESS.tsv', sep="\t", index=False)

In [444]:
tmp_df = pd.merge(le_DOWN_HDAC_ann,DEG_DOWN_HDAC,on='gene_symbol')
tmp_df.to_csv('/home/famgarcia/TFM/LE_DEGs_DOWN_HDAC.tsv', sep="\t", index=False)

In [443]:
tmp_df = pd.merge(le_UP_HDAC_ann,DEG_UP_HDAC,on='gene_symbol')
tmp_df.to_csv('/home/famgarcia/TFM/LE_DEGs_UP_HDAC.tsv', sep="\t", index=False)

# CHECKING LEADING EDGE IN COMMON


In [419]:
common_up_le = list(set(set(le_UP_HDAC['gene_symbol'].to_list()) & set(le_UP_STRESS['gene_symbol'].to_list())))
print("Common up regulated leading edge genes: ", len(common_up_le))

common_down_le = list(set(set(le_DOWN_HDAC['gene_symbol'].to_list()) & set(le_DOWN_STRESS['gene_symbol'].to_list())))
print("Common down regulated leading edge genes: ", len(common_down_le))

up_HDAC_down_Stress = list(set(set(le_UP_HDAC['gene_symbol'].to_list()) & set(le_DOWN_STRESS['gene_symbol'].to_list())))
print("HDAC up regulated leading edge genes, DOWN regulated STRESS LE: ", len(up_HDAC_down_Stress))

down_HDAC_up_Stress = list(set(set(le_DOWN_HDAC['gene_symbol'].to_list()) & set(le_UP_STRESS['gene_symbol'].to_list())))
print("HDAC down regulated leading edge genes, up regulated STRESS LE: ", len(down_HDAC_up_Stress))

Common up regulated leading edge genes:  335
Common down regulated leading edge genes:  140
HDAC up regulated leading edge genes, DOWN regulated STRESS LE:  88
HDAC down regulated leading edge genes, up regulated STRESS LE:  449


In [ ]:
common_up_le = pd.merge(le_UP_HDAC,le_UP_STRESS_ann,on='gene_symbol')

In [427]:
common_down_le = pd.merge(le_DOWN_HDAC,le_DOWN_STRESS_ann,on='gene_symbol')

# DEGs in HDACi common leading edges in selected pathways with stress (ep + ME)

In [450]:
tmp_df_up = pd.merge(common_up_le,DEG_UP_HDAC,on='gene_symbol')

#tmp_df.to_csv('/home/famgarcia/TFM/LE_COMMON_DEGs_UP_HDAC.tsv', sep="\t", index=False)

In [449]:
tmp_df_down = pd.merge(common_down_le,DEG_DOWN_HDAC,on='gene_symbol')

In [451]:
tmp_df = pd.concat((tmp_df_up,tmp_df_down))
tmp_df.to_csv('/home/famgarcia/TFM/LE_common_DEGs_HDAC.tsv', sep="\t", index=False)

# DEGs in Stress common leading edges in selected pathways with stress (ep + ME)

In [452]:
tmp_df_up = pd.merge(common_up_le,DEG_UP_STRESS,on='gene_symbol')
tmp_df_down = pd.merge(common_down_le,DEG_DOWN_STRESS,on='gene_symbol')
tmp_df = pd.concat((tmp_df_up,tmp_df_down))
tmp_df.to_csv('/home/famgarcia/TFM/LE_common_DEGs_STRESS.tsv', sep="\t", index=False)

# CHECKING LEADING EDGE NOT IN COMMON


In [426]:
up_HDAC_down_Stress = pd.merge(le_UP_HDAC,le_DOWN_STRESS_ann,on='gene_symbol')

In [453]:
down_HDAC_up_Stress = pd.merge(le_DOWN_HDAC,le_UP_STRESS_ann,on='gene_symbol')

# DEGs in HDACi in leading edges that are not common in HDAC and ME - effect of epigenetics

In [454]:
tmp_df_up = pd.merge(up_HDAC_down_Stress,DEG_UP_HDAC,on='gene_symbol')
tmp_df_down = pd.merge(down_HDAC_up_Stress,DEG_DOWN_HDAC,on='gene_symbol')

tmp_df = pd.concat((tmp_df_up,tmp_df_down))
tmp_df.to_csv('/home/famgarcia/TFM/LE_notcommon_DEGs_HDAC.tsv', sep="\t", index=False)

# DEGs DOWN in Stress in leading edges that are upregulated in HDAC and ME - effect of ep

In [455]:
tmp_df_down = pd.merge(up_HDAC_down_Stress,DEG_DOWN_STRESS,on='gene_symbol')
tmp_df_up = pd.merge(down_HDAC_up_Stress,DEG_UP_STRESS,on='gene_symbol')

tmp_df = pd.concat((tmp_df_up,tmp_df_down))
tmp_df.to_csv('/home/famgarcia/TFM/LE_notcommon_DEGs_Stress.tsv', sep="\t", index=False)

Saving rest of files

In [380]:
df_annotated.to_csv("/home/famgarcia/TFM/PANTHER_proteins.csv", sep="\t", index=False)

In [399]:
le_UP_HDAC_ann.to_csv("/home/famgarcia/TFM/LE_HDAC_UP_ANNOTATED.csv", sep="\t", index=False)

In [400]:
le_DOWN_HDAC_ann.to_csv("/home/famgarcia/TFM/LE_HDAC_DOWN_ANNOTATED.csv", sep="\t", index=False)

In [401]:
le_UP_STRESS_ann.to_csv("/home/famgarcia/TFM/LE_STRESS_UP_ANNOTATED.csv", sep="\t", index=False)

In [402]:
le_DOWN_STRESS_ann.to_csv("/home/famgarcia/TFM/LE_STRESS_UP_ANNOTATED.csv", sep="\t", index=False)